In [1]:
import pandas as pd
import numpy as np
from collections import Counter
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
# Using scikit models to test againt the data 

In [3]:
train_data = pd.read_csv('train_processed.csv', dtype=str, usecols=range(1,10))
actual = pd.read_csv('gender_submission.csv', usecols=[1], dtype=str)
test_data = pd.read_csv('test_processed.csv', usecols=range(1,9), dtype=str)
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,1,1,0,0,2,1
1,1,1,1,2,1,0,3,0,3
2,1,3,1,1,0,0,1,2,2
3,1,1,1,2,1,0,3,2,3
4,0,3,0,2,0,0,1,2,1


In [4]:
train_X = train_data.drop(['Survived'], axis=1)
# train_X.head()
train_Y = train_data['Survived']
# train_Y.head()
test_X = test_data
# test_X.head()
test_Y = actual
actual.head()

,Survived
0,0
1,1
2,0
3,0
4,1


In [7]:
# one-hot encoding
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc_train_X = enc.fit_transform(train_X)
enc_test_X = enc.fit_transform(test_X)

In [12]:
# KNN with one hot encoding
res = []
for i in range(1, 50):
    knnModel = KNeighborsClassifier(n_neighbors=i)
    knnModel.fit(enc_train_X, train_Y)
    acc_knnModel = round(knnModel.score(enc_test_X, test_Y)*100, 2)
    res.append(acc_knnModel)
#     print(acc_knnModel)
# print('The largest is')
print(max(res))

95.93


In [31]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
classifiers = [
    LogisticRegression(),
    SVC(probability=True),
    KNeighborsClassifier(n_neighbors=30),
    GaussianNB(),
    MultinomialNB(),
    Perceptron(),
    SGDClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier()]
columns = ['Classifier', 'Accuracy']
record = pd.DataFrame(columns = columns)
acc_dict = {}
for cls in classifiers:
    name = cls.__class__.__name__
    cls.fit(train_X, train_Y)
    acc = round(cls.score(test_X, test_Y)*100, 2)
    acc_dict[name] = acc
print(acc_dict.keys())
print(acc_dict.values())


['LogisticRegression', 'KNeighborsClassifier', 'GaussianNB', 'SVC', 'DecisionTreeClassifier', 'RandomForestClassifier', 'SGDClassifier', 'MultinomialNB', 'Perceptron']
[95.93, 91.15, 83.25, 91.15, 83.97, 83.97, 70.33, 84.93, 95.22]


In [15]:
# LogisticRegression (not using ont hot is better)
from sklearn.metrics import accuracy_score
logModel = LogisticRegression()
logModel.fit(train_X, train_Y)
# Y_pred = logModel.predict(test_X)
# print(Y_pred)
acc_logModel = round(logModel.score(test_X, test_Y)*100, 2)
# acc_logModel = accuracy_score(test_Y, Y_pred)
print(acc_logModel)

95.93


In [13]:
# SVM 
svmModel = SVC(probability=True)
svmModel.fit(enc_train_X, train_Y)
acc_svmModel = round(svmModel.score(enc_test_X, test_Y)*100, 2)
print(acc_svmModel)

98.56


In [32]:
# KNN
res = []
for i in range(1, 50):
    knnModel = KNeighborsClassifier(n_neighbors=i)
    knnModel.fit(train_X, train_Y)
    acc_knnModel = round(knnModel.score(test_X, test_Y)*100, 2)
    res.append(acc_knnModel)
#     print(acc_knnModel)
# print('The largest is')
print(max(res))

91.63


In [17]:
# Gaussian Naive Bayes (dense vectors are required)
gaussianModel = GaussianNB()
gaussianModel.fit(train_X, train_Y)
acc_gaussianModel = round(gaussianModel.score(test_X, test_Y) * 100, 2)
print(acc_gaussianModel)

83.25


In [18]:
# Multinomial Naive Bayes (this is comparable with our implemented naive bayers)
MultinomialModel = MultinomialNB()
MultinomialModel.fit(enc_train_X, train_Y)
acc_MultinomialModel = round(MultinomialModel.score(enc_test_X, test_Y) * 100, 2)
print(acc_MultinomialModel)

93.54


In [20]:
# Perceptron (not using one hot is better)
perceptronModel = Perceptron()
perceptronModel.fit(train_X, train_Y)
acc_perceptronModel = round(perceptronModel.score(test_X, test_Y)*100, 2)
print(acc_perceptronModel)

95.22


In [31]:
# Stochastic Gradient Descent (score not stable)
sgdModel = SGDClassifier()
sgdModel.fit(enc_train_X, train_Y)
acc_sgdModel = round(sgdModel.score(enc_test_X, test_Y)*100, 2)
print(acc_sgdModel)

93.06


In [47]:
dict(a = [1,2,3])

{'a': [1, 2, 3]}

In [86]:
# Decision Tree
from sklearn.model_selection import GridSearchCV
# alphas = np.array(range(0,100))/100.
alphas = [0.46,0.1,0.2]
print(alphas)
decisionTreeModel = DecisionTreeClassifier()
Grid = GridSearchCV(estimator=decisionTreeModel, param_grid=dict(min_impurity_split=alphas))
Grid.fit(train_X, train_Y)
# choose the best estimator automatically
acc_decisionTreeModel = round(Grid.score(test_X, test_Y)*100, 2)
print(acc_decisionTreeModel)
print(Grid.best_estimator_.min_impurity_split)

[0.46, 0.1, 0.2]
85.17
0.1


In [91]:
# Decision Tree
decisionTreeModel = DecisionTreeClassifier(min_impurity_split=0.46)
decisionTreeModel.fit(train_X, train_Y)
acc_decisionTreeModel = round(decisionTreeModel.score(test_X, test_Y)*100, 2)
print(acc_decisionTreeModel)

93.78


In [33]:
# Random Forest
random_forestModel = RandomForestClassifier(n_estimators=500)
random_forestModel.fit(train_X, train_Y)
acc_random_forestModel = round(random_forestModel.score(test_X, test_Y)*100, 2)
print(acc_random_forestModel)

83.25


In [87]:
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
# Build a classification task using 3 informative features
X, y = make_classification(n_samples=1000,
                           n_features=10,
                           n_informative=3,
                           n_redundant=0,
                           n_repeated=0,
                           n_classes=2,
                           random_state=0,
                           shuffle=False)


rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, oob_score = True) 

param_grid = { 
    'n_estimators': [200, 700],
    'max_features': ['auto', 'sqrt', 'log2']
}

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(train_X, train_Y)
print CV_rfc.best_params_

/Users/apple/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/apple/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


{'max_features': 'auto', 'n_estimators': 700}


In [94]:
acc_random_forestModel = round(CV_rfc.score(test_X, test_Y)*100, 2)
print(acc_random_forestModel)
print(CV_rfc.best_params_)

83.97
{'max_features': 'auto', 'n_estimators': 700}


In [92]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes Gussian', 'Perceptron', 
              'Stochastic Gradient Decent', 'Multinomial Naive Bayers', 
              'Decision Tree'],
    'Score': [acc_svmModel, acc_knnModel, acc_logModel, 
              acc_random_forestModel, acc_gaussianModel, acc_perceptronModel, 
              acc_sgdModel, acc_MultinomialModel, acc_decisionTreeModel]})
models.sort_values(by='Score', ascending=False)

,Model,Score
0,Support Vector Machines,98.56
2,Logistic Regression,95.93
1,KNN,95.69
5,Perceptron,95.22
8,Decision Tree,93.78
7,Multinomial Naive Bayers,93.54
6,Stochastic Gradient Decent,93.06
3,Random Forest,83.97
4,Naive Bayes Gussian,83.25
